<a href="https://colab.research.google.com/github/jjonhwa/Hands-On-Machine-Learning/blob/main/Chapter_7_%EC%95%99%EC%83%81%EB%B8%94_%ED%95%99%EC%8A%B5%EA%B3%BC_%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**앙상블**  

일련의 예측기로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있다. 이 때 일련의 예측기를 앙상블이라고 부른다.

가장 간단한 예로, Training Set에서 무작위로 **각기 다른 서브셋**을 만들어 일련의 결정트리 분류기를 훈련시킬 수 있다.  


**배깅, 부스팅, 스태킹**에 대하여 알아보도록 하자.

# 7.1 투표 기반 분류기

더 좋은 분류기를 만드는 매우 간단한 방법은 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것입니다. 이를 **직접 투표 분류기**라고 한다.  

흔히, 다수결 투표 분류기가  가장 뛰어난 것보다도 정확도가 높을 경우가 많다.  
어떻게 이렇게 될 수가 있을까??  

동전 앞면이 51%. 뒷면이 49%가 나오는 동전이 있다고 할 때, 1000번 던진 후 앞면이 다수가 될 확률은 75%에 가깝다. 이는 **큰 수의 법칙**때문이다. 더 많이 던질수록 앞면이 나오는 비율은 더 올라간다.  

* **큰 수의 법칙** : Binomial Distribution의 pmf를 계산하여 얻을 수 있다.  
성공 확률이 51%인 동전을 1000번 던져서 앞면이 50번 이상 나올 확률 : ${\frac{n}{k}}p^k (1-p)^{(n-k)}$  
위의 식을 이용하여 1~499까지 더한 후 1에서 뺴면 그 값이 앞면이 다수가 될 확률이다.

* 이처럼 51% 정확도를 가진 1000개의 분류기로 앙상블 모델을 구축한다고 할 때 가장 많은 클래스를 예측으로 삼는다면 75%의 정확도를 기대할 수 있는 것이다.

**Note :** 이런 가정은 모든 분류기가 완벽하게 독립적이고 오차에 상관관계가 없어야 가능합니다.

**Tip :** 앙상블 방법은 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘합니다. 다양한 분류기를 얻는 한 가지 방법은 **각기 다른 알고리즘**으로 학습시키는 것이다. 이럴 때 앙상블 모델의 정확도가 향상됩니다.  


다음은 투표 기반 분류기(`VotingClassifier`)를 만들고 훈련시키는 코드이다. 확인해보도록 하자.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = 'hard'
)

voting_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf) :
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)

  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

현재 페이지에 X, y를 정의해주지 않아서, 코드를 실행하지는 않고 확인만 하고 넘어가도록 하자.

위의 코드의 경우 "voting = 'hard'"로 주면 분류한 것들의 개수를 가지고 최종 선택을 진행하며 "voiting = 'soft'"로 주면 각 분류기의 예측확률을 평균 내어 가장 높은 클래스를 예측한다.(모든 분류기가 확률을 추정할 수 있어야 한다. SVC의 경우 default로 확률을 제공하지 않으므로 probability 매개변수를 True로 지정해야한다.) 각각을 **Hard Voting**, **Soft Voting**이라고 한다.

# 7.2 배깅과 페이스팅

다양한 분류기를 만드는 다른 방법은 **같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습**시키는 것이다.

* 배깅 : Bootstrap Aggregating의 줄임말로, 훈련 세트의 중복을 허용하여 샘플링하는 방식이다.
* 페이스팅 : 중복을 허용하지 않고 샘플링하는 방식이다.

즉, 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있다는 것을 의미한다.

* 수집 함수 : 최종으로 여러 예측기를 합칠 때 쓰는 함수를 뜻하며, 전형적으로 분류일 떄는 최빈값을 회귀일 떄는 평균을 계산한다.

일반적으로 각 예측기는 크게 편향되어 있지만 수집 함수를 통과하면 편향과 분산이 모두 감소합니다. 일반적인 앙상블의 결과는 편향은 비슷하지만 분산이 줄어드는 것을 볼 수 있습니다.

**Note :** 배깅과 페이스팅의 인기가 높은 이유는 예측기들이 모두 동시에 다른 CPU 코어나 서버에서 병렬로 학습이 가능하기 때문입니다.


## 7.2.1 사이킷런의 배깅과 페이스팅

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    max_samples = 100, bootstrap = True, n_jobs = -1
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

* `DecisionTreeClassifier()` : DecisionTree 분류기 사용  
* `n_estimators` : 500개의 트리를 앙상블 훈련  
* `max_samples` : 100개의 샘플을 선택  
* `bootstrap` : True면 배깅, False면 페이스팅으로  True이므로 샘플을 중복허용 무작위 선택한다.(False면 중복을 허용하지 않고 무작위 선택)
* `n_jobs` : 사이킷런의 훈련과 예측에 사용할 CPU 코어 수를 지정하고 '-1'로 지정하면 가용한 모든 코어를 사용한다.

**Note :** 각 분류기들이 확률을 추정할 수 있으면 자동으로 간접 투표 방식을 사용합니다.

일반적으로 배깅이 페이스팅보다 편향이 조금 높다.(서브셋의 다양성을 증가시키므로) 하지만 다양성을 높이는 것은 상관관계를 줄이므로 앙상블의 분산을 감소한다. 그래서 전반적으로 배깅이 페이스팅보다 더 선호된다.

## 7.2.2 oob 평가

배깅을 사용하면(중복 허용) 어떤 샘플을 각 예측기에 여러번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있다. 
* 이는 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링 된다는 것을 의미한다. 이 떄 남겨진 37%를 **oob 샘플**이라고 한다.
  * 샘플 수 m이 커짐에 따라 $1 - exp(-1)$ 즉, 63.212%에 가까워진다. (m개의 샘플에서 무작위로 하나를 선택할 때 선택되지 않을 확률은 $1-\frac{1}{m}$이고 이를 m번 반복하면 $(1-\frac{1}{m})^m$이므로 m이 커질수록 로피탈 정리에 의하여 $e^-1$이 된다.
* oob샘플을 훈련할 때 사용하지 않으므로 oob 샘플을 Validation Set으로 사용할 수 있다.
* `BaggingClassifier`를 만들 때 `oob_score = True`로 지정하면 oob 평가를 자동으로 수행한다. 그 점수는 `oob_score_`변수에 저장되어 있다.

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    bootstrap = True, n_jobs = -1, oob_score = True
)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

분류기들처럼 oob 샘플에 대한 결정 함수의 값도 `oob_decision_function_` 변수를 사용하여 확인할 수 있다.

In [ ]:
bag_clf.oob_decision_function_

# 7.3 램덤 패치와 랜덤 서브스페이스

* 랜덤 패치 방식 : 훈련 특성과 샘플을 모두 샘플링하는 방식
* 랜덤 서브스페이스 방식 : 훈련 샘플은 모두 사용하고  특성은 샘플링 하는 방식
* `max_samples`, `bootstrap`, `max_features`, `bootstrap_features`를 활용하여 샘플링을 조절한다. 앞 두 개는 Observation에 대한 샘플링이고 뒤의 두 개는 Feature에 대한 샘플링이다.

Feature 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춘다.

# 7.4 랜덤 포레스트
랜덤 포레스트는 일반적인 배깅 방법을 적용한 Decision Tree의 Ensenble이다.

* `max_samples` : 훈련 세트의 크기 지정.
* BaggignClassifier에 DecisionTreeClassifier을 넣어 만드는 모양으로 이렇게 만드필요는 없으며 `RandomForestClassifier`이라는 매서드를 사용할 수 있다.(앞의 매서드는 분류메서드이며 회귀의 경우 `RandomForestRegressor`이 있다.)
* RandomForest 알고리즘은 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입한다. 이로 인해, 편향을 손해보지만 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어낸다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators = 500,
                                 max_leaf_nodes = 16,
                                 n_jobs = -1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

RandomForestClassifier의 경우 몇 가지 예외가 있긴 하지만 DecisionTreeClassifier가 가지는 매개변수와 BaggingClassifier가 가지는 매개변수를 모두 가지고 있다.(예외 항목 : {'splitter' : best, 'presort' : False, 'max_samples' : 1.0, 'base_estimator' : 무조건 지정된 매개변수를 사용한 DecisionTree}

다음은 RandomForestClassifier을 BaggingClassifier를 활용해 나타낸 것이다. 확인만 하고 넘어가도록 하자.

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features = 'auto', max_leaf_nodes = 16),
    n_estimators = 500,
    max_samples = 1.0,
    bootstrap = True,
    n_jobs = -1
)

## 7.4.1 엑스트라 트리

* Extra-Trees :RandomForest에서 Feature의 subset을 만들어 분할할 떄 최적의 임곗값는 대신 후보 특성을 사용ㅎ해 무작위로 분할한 다음 그 중에서 최상의 분할을 선택한다. (이 역시, 편향이 늘고 분산을 낮추게된다.)
* 흔히 최적의 임곗값을 찾기위해 시간을 많이 소요하는데 여기서는 최적의 임곗값을 찾지 않으므로 시간이 훨씬 단축되며 RandomForest보다 훨씬 빠르다.
* `ExtraTreesClassifier` 메서드를 사용하며 사용법은 RandomForestClassifier과 같다.

**Tip :** Extra-Tree와 RandomForest 중에서 뭐가 더 좋을지는 섯불리 예측하기는 힘들다. 일반적으로 둘 다 시도해보고 교차 검증을 통해 비교하는 것이 유일한 방법입니다.

## 7.4.2 특성 중요도


* 랜덤포레스트의 장점으로 특성의 상대적 중요도를 측정하기 쉽다는 것이다.(DecisionTree 기반 모델은 모두 특성 중요도를 제공한다.)
* 평균적으로 불순도를 얼마나 감소시키는지에 따라서 특정 중요도를 측정한다.
* 랜덤 포레스트이 특성 중요도는 각 결정 트리의 특성 중요도를 모두 계산하여 더한 후 트리 수로 나눈 것이다.
* 특성 중요도는 `feature_importances_` 변수에 저장되어 있다.

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)
rnd_clf.fit(iris['data'], iris['target'])

for name, score in zip(iris['feature_names'], rnd_clf.feature_importnaces_) :
  print(name, score)

랜덤 포레스트는 특히 *특성을 선택해야 할 때 어떤 특성이 중요한지 빠르게 확인할 수 있어 매우 편리하다.

# 7.5 부스팅
부스팅 : 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법.  
아이디어 : 앞의 모델을 보완해나가면서 일련의 예측기를 학습시키는 것.

* 에이다부스트(AdaBoost)

**Note :** 이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것이다. 이렇게 하면 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰지게 된다.

1. 첫 번째 분류기를 훈련시키고 예측.
2. 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높인다.
3. 업데이트된 가중치를 사용하여 두 번째 분류기에서 훈련하고 예측한다.
4. 위의 일련의 과정을 반복한다.

에이다 부스트는 Cost를 최소화 하는 것이 아닌 점점 좋아지도록 앙상블에 예측기를 추가하는 방식으로 학습한다.

**Caution :** 에이다 부스트와 같은 연속된 학습 기법에서는 병렬화가 될 수 없기 때문에 확장성이 안좋다는 단점이 있다.

* 에이다부스트 알고리즘

1. 각 샘플 가중치 $w^{(i)}$는 초기에 모두 $\frac{1}{m}$로 초기화 된다.
2. 첫 번째 예측기가 학습되고, 가중치가 적용된 에러율 $r_1$이 훈련 세트에 대해 계산된다.
3. 에러율 : 잘못 예측된 예측기에서의 가중치합 / 전체 가중치합
4. 예측기 가중치($\alpha_j$) = $\eta log{\frac{1-r_j}{r_j}}$ (이떄, $\eta$는 학습률 하이퍼파라미터이다.)
5. 예측기가 정확할수록 가중치가 높다. 무작위 예측이라면 가중치가 0에 가깝다. 무작위 추측보다 정확도가 낮다면) 가중치는 음수가 된다.
6. 가중치 업데이트 규칙 : $w^{(i)}$ = $\begin{cases}
w^{(i)} & \text{if } \hat{{y_j}^{(i)}} = y^{(i)}\text{ 일 때} \\
w^{(i)}exp(\alpha_j) & \text{if } \hat{{y_j}^{(i)}} \neq y^{(i)}\text{ 일 때}\\
\end{cases}$
7. 가중치를 업데이트 한 후 모든 샘플의 가중치를 가중치 총 합으로 나누어 정규화한다.
8. 마지막으로 새 예측기가 업데이트된 가중치를 사용해 훈련되고 이 일련의과정이 반복된다.
9. 지정된 예측기 수 혹은 완벽한 예측기가 만들어지면 중지한다.
10. 단순히 모든 예측기의 예측을 계산하고 예측기 가중치 $\alpha_j$를 더해 예측 결과를 만든다. 이 때 가중치 합이 가장 큰 클래스가 예측 결과가 된다.
11. 에이다부스트의 다중 클래스 버전인 `SAMME`가 있다. 클래스가 두 개 뿐일 때는 에이다부스트와 동일하다.
12. 앞서 배운 'soft'처럼 확률을 구할 수 있다. 이 떄는 `SAMME.R`을 사용한다.


다음은 `AdaBoostClassifier`를 사용하여 200개의 아주 얕은 결정 트리 기반 에이다부스트분류기를 훈련시키는 코드이다. 확인하고 넘어가도록 하자.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1),
    n_estimators = 200,
    algorithm = 'SAMME.R',
    learning_rate = 0.5
)

ada_clf.fit(X_train, y_train)

**Tip :** 에이다부스트 앙상블이 훈련 세트에 과대적합이된다면 **추정기 수를 줄이거나** 추정기의 **규제**를 더 강하게 해보세요.

## 7.5.2 그레디언트 부스팅
* 그레디언트 부스팅 : 에이다부스터와 비슷하며 예측기를 추가할 때 오차를 보정하면서 예측기를 수정한다.(가중치를 수정하는 대신 잔여 오차에 대한 새로운 예측기를 학습한다.)

다음은 그레디언트 부스티드 회귀 트리(GBRT)이다. 확인해보도록 하자.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth = 2)
tree_reg1.fit(X, y)

위의 GBRT에서 생긴 잔여 오차에 대하여 두 번째 DecisionTreeRegressor를 훈련시킨다.

In [ ]:
y2 = y - tree_reg1.predict(X)

tree_reg2 = DecisionTreeRegressor(max_depth = 2)
tree_reg2.fit(X, y2)

In [ ]:
y3 = y2 - tree_reg2.predict(X)

tree_reg3 = DecisionTreeRegressor(max_depth = 2)
tree_reg3.fit(X, y3)

위의 코드들을 바탕으로 3개의 트리를 포함하는 Ensenble 모델이 생겼다. 새로운 샘플에 대한 예측을 하기 위해서는 **모든 트리의 예측을 더하면 된다.**


In [ ]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

위와 같은 일련의 과정을 `GradientBoostingRegressor`을 사용하면 GBRT 앙상블을 간단하게 훈련시킬 수 있다. RandomForestRegressor와 비슷한 매개변수를 가지고 있다.(ex, `n_estimators`, `max_depth`, `min_samples_leaf` etc)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 3, learning_rate = 1.0)
gbrt.fit(X, y)

`learning_rate` 매개변수가 각 트리의 기여 정도를 조절한다.
- 낮게 설정하면 많은 트리가 필요하지만 예측 성능은 좋아진다.(규제방법 : **축소**라고 한다. 너무 줄일 경우 Overfit의 위험이 있음)


최적의 트리 수를 찾기 위해서 **조기 종료**를 사용할 수 있다. `staged_predict()`메서드를 사용하면 사용할 수 있다.  
다음의 코드를 확인해보도록 하자.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimaotrs = 120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
  for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(erros) + 1

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators = bst_n_estimators)
gbrt_best.fit(X_train, y_train)

조기 종료를 사용할 수 있는 또 다른 방법으로는  `warm_start = True`로 설정해주는 것이다. 이것은 사이킷런의 `fit()` 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 해준다. 즉, 기존의 트리를 유지하면서 다음 트리를 이용했을 때 검증 오차가 향상되지 않으면 훈련을 멈추게한다.(몇 번까지 오르지 않을 건지는 입력값으로 준다.)

In [ ]:
gbrt = GradientBoostingRegressor(max_depth = 2, warm_start = True)

min_val_error = float('inf')
error_going_up = 0

for n_estimators in range(1, 120) :
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)

  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)

  if val_error < min_val_error :
    min_val_error = val_error
    error_going_up = 0
  else : 
    error_going_up += 1
    if error_going_up = 5 :
      break

* 확률적 그레디언트 부스팅 : GradientBoostingRegressor에서 각 트리가 훈련할 떄 사용할 훈련 샘플의 비율을 지정한 것을 확률적 그레디언트 부스팅이라고 한다. 이것 역시 편향이 높아지는 대신 분산이 낮아지며 샘플을 사용하는만큼 훈련 속도가 상당히 높아집니다.

**Note :** GradientBoosting에서 MSE 뿐만 아니라 다른 Cost Function을 이용할 수도 있습니다. 필요에 따라 확인하여 사용하도록 하자.

* XGBoost : 최적화된 그레디언트 부스팅 구현으로 익스트림 그레디언트 부스팅의 약자이다. 이 패키지는 매우 빠른 속도, 확장성, 이식성을 목표로 만들어 졌으며 우승 후보들이 사용하는 중요한 도구 중에 하나이다. 꼭 깊이있게 알아보도록 하자. XGBoost API는 사이킷런과 매우 비슷하다. 
* XGBoost는 자동 조기 종ㄹ료와 같은 여러 좋은 기능도 제공한다. 확인해보도록 하자.

간단한 다음의 코드를 진행해보고 마무리하도록 하자.

In [ ]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [ ]:
xgb_reg.fit(X_train, y_train,
            eval_set = [(X_val, y_val)], early_stopping_rounds = 2)
y_pred = xgb_reg.predict(X_val)

# 7.6 스태킹
* 아이디어 : 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까요?라는 질문으로부터 파생되었다.
* 각기 다른 값을 예측한 후에 마지막 예측기(**블렌더**)를 사용하여 최종 예측을 만든다.
* 일반적으로 블렌더는 Hold-Out세트를 사용하여 학습시킨다.
* 스태킹 알고리즘
1. 먼저 훈련 세트를 두 개의 서브셋으로 나눈다.
2. 첫 번째 서브셋으로 첫 번째 레이어의 예측기를 훈련시킨다.
3. 첫 번째 레이어의 에측기를 사용하여 두 번째 서브셋에 대한 예측 결과를 반환한다.(첫 번째 서브셋이 Training Set, 두 번째 서브셋이 Test Set 같은 느낌)
4. 타깃값은 그대로 쓰고 예측한 값을 입력 특성으로 사용하는 새로운 Training Set을 만든다.(즉 새로운 Training Set이 블렌더로서 새 훈련 세트가 된다.)


* 여러 개의 블렌더를 사용하는 것도 가능하다.(여러 층의 Layer 사용)
* 안타깝게도 사이킷런은 스태킹을 직접 지원하지 않는다. 직접 구현을 해야하며 다행히 직접 구현하는 것은 어렵지 않다. [여기](https://github.com/Menelau/DESlib)에서 오픈 소스 구현을 사용할 수 있다. 참고하도록 하자.